In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime as dt

In [27]:
# this function converts the range of operational days to a list of daysofweek numbers
def get_days_of_week(days):
    days_words_to_num_mapping = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}
    days_in_numbers = []
    day_ranges_list = days.split(',')
    for day_range in day_ranges_list:
        days_in_words = day_range.split('-')
        if len(days_in_words) < 1:
            return None
        elif len(days_in_words) == 1:
            days_in_numbers.append(days_words_to_num_mapping[days_in_words[0].replace(" ","")])
        else:
            days_in_numbers.extend(range(days_words_to_num_mapping[days_in_words[0].replace(" ","")], days_words_to_num_mapping[days_in_words[1].replace(" ","")]))
            days_in_numbers.append(days_words_to_num_mapping[days_in_words[1].replace(" ","")])
    return days_in_numbers

In [35]:
def get_preprocessed_df(restaurant_sheet):
#     separating multiple, 'slash-separated' timings for a single restaurant
    restaurant_sheet.operational_timing=restaurant_sheet.operational_timing.str.split('/')
    processed_df = restaurant_sheet.explode('operational_timing')
    
#     extracting the operational days from the time field
    processed_df['days'] = processed_df.operational_timing.str.replace('\d{1,2}(?::\d{2})? [ap]m - \d{1,2}(?::\d{2})? [ap]m', '', regex = True)
    processed_df.days = processed_df.days.str.strip()
    
#     extracting the range(s) of operational days from the time field 
    processed_df['timings'] = [a.replace(b, '').strip() for a, b in zip(processed_df['operational_timing'], processed_df['days'])]
    processed_df["timings"] = processed_df.apply(lambda x: x["operational_timing"].replace(x["days"], "").strip(), axis=1)
    
#     extracting the opening and closing hours from the time field
    processed_df[['opens_from', 'closes_at']] = processed_df.timings.apply(lambda x: pd.Series(str(x).split("-")))
    
#     extracting the operational days as list of daysofweek numbers
    processed_df['days_in_numbers'] = processed_df['days'].apply(get_days_of_week)
    processed_df.opens_from = processed_df.opens_from.str.strip()
    processed_df.closes_at = processed_df.closes_at.str.strip()
    
#     converting the opening and closing hours to 24 hour format
    processed_df['opens_from_24h'] = pd.to_datetime(processed_df['opens_from'], format = '%I:%M %p', errors="coerce").fillna(pd.to_datetime(processed_df['opens_from'], format="%I %p", errors="coerce")).dt.time
    processed_df['closes_at_24h'] = pd.to_datetime(processed_df['closes_at'], format = '%I:%M %p', errors="coerce").fillna(pd.to_datetime(processed_df['closes_at'], format="%I %p", errors="coerce")).dt.time
    print("Input data, post preprocessing..")
    display(processed_df)
    return processed_df

In [37]:
def get_open_restaurants(filename, time_to_check):
#     reading excel file with explicitly defined header  
    column_names = ['restaurants','operational_timing']
    restaurant_vs_timing = pd.read_excel(filename, header=None, names = column_names)
    
#     preprocessing operational timing field to simplify the values
    preprocessed_df = get_preprocessed_df(restaurant_vs_timing)

#     processing the input time of enquiry
    day_to_check = pd.to_datetime(time_to_check, format='%Y-%m-%d %H:%M:%S').dayofweek
    time_of_enquiry = pd.to_datetime(time_to_check, format='%Y-%m-%d %H:%M:%S').time()
    
#     fetching list of restaurants operational at given time
    preprocessed_df['day_filter'] = preprocessed_df['days_in_numbers'].apply(lambda x: day_to_check in x)
    nested_list_of_restaurants = preprocessed_df[['restaurants']][(preprocessed_df['day_filter']==True) & (time_of_enquiry<preprocessed_df['closes_at_24h']) & (time_of_enquiry>preprocessed_df['opens_from_24h'])].values.tolist()
    print("List of restuarants operational at given time..")
    return [item for sublist in nested_list_of_restaurants for item in sublist]

In [38]:
filename = 'restaurants-opening-hour.xlsx'

In [39]:
get_open_restaurants(filename, '2023-01-24 16:17:48')

Input data, post preprocessing..


,restaurants,operational_timing,days,timings,opens_from,closes_at,days_in_numbers,opens_from_24h,closes_at_24h
0,Kushi Tsuru,Mon-Sun 11:30 am - 9 pm,Mon-Sun,11:30 am - 9 pm,11:30 am,9 pm,"[0, 1, 2, 3, 4, 5, 6]",11:30:00,21:00:00
1,Osakaya Restaurant,"Mon-Thu, Sun 11:30 am - 9 pm","Mon-Thu, Sun",11:30 am - 9 pm,11:30 am,9 pm,"[0, 1, 2, 3, 6]",11:30:00,21:00:00
1,Osakaya Restaurant,Fri-Sat 11:30 am - 9:30 pm,Fri-Sat,11:30 am - 9:30 pm,11:30 am,9:30 pm,"[4, 5]",11:30:00,21:30:00
2,The Stinking Rose,"Mon-Thu, Sun 11:30 am - 10 pm","Mon-Thu, Sun",11:30 am - 10 pm,11:30 am,10 pm,"[0, 1, 2, 3, 6]",11:30:00,22:00:00
2,The Stinking Rose,Fri-Sat 11:30 am - 11 pm,Fri-Sat,11:30 am - 11 pm,11:30 am,11 pm,"[4, 5]",11:30:00,23:00:00
...,...,...,...,...,...,...,...,...,...
49,Kyoto Sushi,Mon-Thu 11 am - 10:30 pm,Mon-Thu,11 am - 10:30 pm,11 am,10:30 pm,"[0, 1, 2, 3]",11:00:00,22:30:00
49,Kyoto Sushi,Fri 11 am - 11 pm,Fri,11 am - 11 pm,11 am,11 pm,[4],11:00:00,23:00:00
49,Kyoto Sushi,Sat 11:30 am - 11 pm,Sat,11:30 am - 11 pm,11:30 am,11 pm,[5],11:30:00,23:00:00
49,Kyoto Sushi,Sun 4:30 pm - 10:30 pm,Sun,4:30 pm - 10:30 pm,4:30 pm,10:30 pm,[6],16:30:00,22:30:00


List of restuarants operational at given time..


['Kushi Tsuru',
 'Osakaya Restaurant',
 'The Stinking Rose',
 "McCormick & Kuleto's",
 'Mifune Restaurant',
 'The Cheesecake Factory',
 'New Delhi Indian Restaurant',
 'Iroha Restaurant',
 'Rose Pistola',
 "Alioto's Restaurant",
 'Canton Seafood & Dim Sum Restaurant',
 'All Season Restaurant',
 'Bombay Indian Restaurant',
 "Sam's Grill & Seafood Restaurant",
 '2G Japanese Brasserie',
 'Restaurant Lulu',
 'Herbivore',
 'Penang Garden',
 "John's Grill",
 'Quan Bac',
 'Burger Bar',
 'Blu Restaurant',
 'Shanghai China Restaurant',
 'Tres',
 'Isobune Sushi',
 'Far East Cafe',
 'Parallel 37',
 'Bai Thong Thai Cuisine',
 'Alhamra',
 "Nick's Lighthouse",
 'Paragon Restaurant & Bar',
 'Chili Lemon Garlic',
 'Bow Hon Restaurant',
 'San Dong House',
 "Cesario's",
 'Colombini Italian Cafe Bistro',
 'Sabella & La Torre',
 'Soluna Cafe and Lounge',
 'Tong Palace',
 'India Garden Restaurant',
 'Sapporo-Ya Japanese Restaurant',
 "Santorini's Mediterranean Cuisine",
 'Kyoto Sushi']